In [1]:
import requests
import re
from bs4 import BeautifulSoup
import csv
import random
import time
import numpy as np

In [2]:
pages = 1 # number of pages to scrape
distance = 3 # search radius in km within an area

#link = "https://vancouver.craigslist.org/d/apts-housing-for-rent/search/apa"
path = "pages/page_"
items=120

In [3]:
# USER AGENTS

# Checkout https://developers.whatismybrowser.com/useragents/explore/

useragents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
    'Mozilla/5.0 (Linux; U; Android 4.1.2; de-de; GT-I8190 Build/JZO54K) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30',
    'Mozilla/5.0 (Linux; U; Android 4.1.2; de-de; ME371MG Build/JZO54K) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Safari/534.30',
    'Mozilla/5.0 (Linux; U; Android 4.2.2; pl-pl; GT-P5110 Build/JDQ39) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Safari/534.30',
    'Mozilla/5.0 (Linux; U; Android 4.1.1; de-de; SGPT12 Build/TJDSU0177) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Safari/534.30',
    'Opera/9.80 (J2ME/MIDP; Opera Mini/7.1.32052/29.3417; U; en) Presto/2.8.119 Version/11.10',
    'Opera/9.80 (J2ME/MIDP; Opera Mini/7.1.32052/29.3594; U; en) Presto/2.8.119 Version/11.10',
    'Opera/9.80 (J2ME/MIDP; Opera Mini/4.2.23449/28.2144; U; en) Presto/2.8.119 Version/11.10',
    'Opera/9.80 (J2ME/MIDP; Opera Mini/4.2.20464/28.3950; U; en) Presto/2.8.119 Version/11.10',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1',
    'Mozilla/5.0 (Windows NT 5.1; rv:36.0) Gecko/20100101 Firefox/36.0',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:43.0) Gecko/20100101 Firefox/43.0',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:57.0) Gecko/20100101 Firefox/57.0	',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) Gecko/20100101 Firefox/63.0',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:59.0) Gecko/20100101 Firefox/59.0',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:45.0) Gecko/20100101 Firefox/45.0',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 11_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/601.7.7 (KHTML, like Gecko) Version/9.1.2 Safari/601.7.7',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.1.2 Safari/603.3.8',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15'    
]

In [4]:
# Free Proxies

#https://free-proxy-list.net/

def get_proxies():
    proxies =[]
    url = 'https://free-proxy-list.net/'
    r = requests.get(url)
    s = BeautifulSoup(r.content,'html.parser')
    dt = s.find_all("tr", limit=21)
    for i in range(20):
        if dt[i+1].find_all("td")[4].text == "elite proxy":
            px = dt[i+1].find_all("td")[0].text + ":" + dt[i+1].find_all("td")[1].text
            proxies.append(px)
    return proxies



In [5]:

# RUN IT IF YOU WANT TO SAVE WEBPAGES

# for i in range(pages):
#     fname = path+str(i+1)+".txt"
#     r = requests.get(link)
#     open(fname, 'wb').write(r.content)
#     #s = BeautifulSoup(r.content,'html.parser')
#     #dt = s.find_all("li","result-row")
#     link = "https://vancouver.craigslist.org/search/apa?s="+str(items*(i+1))
    
    
# def get_lat_lon(url,proxies,useragents):
    
#     user_agent = random.choice(useragents)
#     headers = {'User-Agent': user_agent}
#     proxy = random.choice(proxies)
#     mapdt = requests.get(url,headers=headers,proxies={"http": proxy, "https": proxy})
#     ss = BeautifulSoup(mapdt.content,'html.parser')
#     dd = ss.find("div","viewposting")
#     lat = dd.get("data-latitude")
#     long = dd.get("data-longitude")
#     return lat,long


In [6]:
def get_postal_codes():
    
    area = []
    aname = []
    l = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V"
    r = requests.get(l)
    s = BeautifulSoup(r.content,'html.parser')
    dt = s.find_all("td",limit = 180)
    for a in dt:
        area.append(a.find("b").text)
        names = a.find_all("a")
        n = ""
        for na in names:
            n = n + na.text+", "
        n = n.strip(" ,")
        aname.append(n)
    return area,aname

In [7]:

with open('rental_data.csv', 'w',newline='') as csvfile:
    
    filewriter = csv.writer(csvfile, delimiter=',')
    filewriter.writerow(['Post_ID', 'Time', 'House_type', 'Area', 'Area_Name', 'Address','Rent','url'])
    #filewriter.writerow(['Post_ID', 'Time','lat','long', 'House_type', 'Address','Rent','url'])
    

    proxies = get_proxies()
    areas,area_names = get_postal_codes()
    areas = ['T5T', 'V5A', 'V3E','V7A']
    area_names = ['',
                  'Burnaby, Government Road, Lake City, SFU, Burnaby Mountain',
                  'Coquitlam',
                  'Richmond']
    
    st = np.random.randint(2, 10, size=len(areas))
    updateproxy = np.random.randint(0, 2, size=len(areas))

    for x in range(len(areas)):
        
        if area_names[x] == "":
            continue
        
        link = "https://vancouver.craigslist.org/search/apa?search_distance="+str(distance)+"&postal="+areas[x]+"&availabilityMode=0&sale_date=all+dates"
         
        if updateproxy[x]:
            proxies = get_proxies() 
        
        user_agent = random.choice(useragents)
        headers = {'User-Agent': user_agent}
        proxy = random.choice(proxies)
        
        #print(str(x) +"<-Loop Sleep->"+str(st[x]))#############
        time.sleep(st[x])
        i = 0
        total = 120
        
        while True:
    
    
            i = i+1
            print("Scraping rental properties in "+area_names[x]+" region(s) -> Page "+str(i))
            
            while True:
                try:
                    r = requests.get(link,headers=headers,proxies={"http": proxy, "https": proxy})
                    if r.status_code <400:
                        print("-> Scrape successful.")
                        break
                except:
                    print("-> Scrape unsuccessful. Retrying...")
                    time.sleep(random.randint(0,7))
                    proxy = random.choice(proxies)

            s = BeautifulSoup(r.content,'html.parser')
            dt = s.find_all("li","result-row")

            # SCRAPE DATA

            for item in dt:
                post_id = item.find("span", "maptag").get("data-pid")
                url = item.find("a").get("href")
                tm = item.find("time").get("datetime")
                house_type = ""
                address = ""
                rent = ""
                htb = ""
                hta = "" 

                if item.find("span","housing") is not None:
                    ht = item.find("span","housing").text.replace("\n","").replace(" ", "")

                if item.find("span", "result-hood") is not None:
                    a = item.find("span", "result-hood").text.strip()
                    address = re.sub('[()]', '',a)

                if item.find("span","result-price") is not None:
                    rent = item.find("span","result-price").text

                filewriter.writerow([post_id,tm,ht,areas[x],area_names[x],address,rent,url])
                
            if total<int(s.find("span","totalcount").text):
                
                link = "https://vancouver.craigslist.org/search/apa?s="+str(total)+"&availabilityMode=0&postal="+areas[x]+"&search_distance="+str(distance)
                total = total+120
                continue
                
            break

        
        


Scraping rental properties in Burnaby, Government Road, Lake City, SFU, Burnaby Mountain region(s) -> Page 1
-> Scrape successful.
Scraping rental properties in Coquitlam region(s) -> Page 1
-> Scrape successful.
Scraping rental properties in Richmond region(s) -> Page 1
-> Scrape successful.


In [8]:



# with open('rental_data.csv', 'w',newline='') as csvfile:
    
#     filewriter = csv.writer(csvfile, delimiter=',')
#     filewriter.writerow(['Post_ID', 'Time', 'House_type', 'Address','Rent','url'])
#     #filewriter.writerow(['Post_ID', 'Time','lat','long', 'House_type', 'Address','Rent','url'])
    
#     st = np.random.randint(2, 10, size=pages)
#     proxies = get_proxies()
    
#     for i in range(pages):
        
        
#         if i==3:
#             proxies = get_proxies() 
        
#         user_agent = random.choice(useragents)
#         headers = {'User-Agent': user_agent}
#         proxy = random.choice(proxies)
        
#         time.sleep(st[i])
    
#         # READ FROM FILE
    
# #         fname = path+str(i+1)+".txt"
# #         f = open(fname, 'r',encoding="utf8")
# #         s = BeautifulSoup(f,'html.parser')
# #         dt = s.find_all("li","result-row")
    
#         # READ DIRECTLY FROM WEB PAGE
    
#         while True:
#             try:
#                 r = requests.get(link,headers=headers,proxies={"http": proxy, "https": proxy})
#                 if r.status_code <400:
#                     break
#             except:
#                 proxy = random.choice(proxies)
            
#         s = BeautifulSoup(r.content,'html.parser')
#         dt = s.find_all("li","result-row")
#         link = "https://vancouver.craigslist.org/search/apa?s="+str(items*(i+1))
    
#         # SCRAPE DATA
    
#         for item in dt:
#             post_id = item.find("span", "maptag").get("data-pid")
#             url = item.find("a").get("href")
#             tm = item.find("time").get("datetime")
#             house_type = ""
#             address = ""
#             rent = ""
#             htb = ""
#             hta = ""
            
#             # Get Latitude and Longitude
#             #time.sleep(random.randint(29,37))
#             #lat,long = get_lat_lon(url,proxies,useragents)
 
            
#             if item.find("span","housing") is not None:
#                 ht = item.find("span","housing").text.replace("\n","").replace(" ", "")
# #                 vals = re.findall('\d+', str(ht))
# #                 if len(vals)==2:
# #                     htb = vals[0]
# #                     hta = vals[1]
# #                 elif len(vals)==1:
# #                     htb = vals[0]
                
#             if item.find("span", "result-hood") is not None:
#                 a = item.find("span", "result-hood").text.strip()
#                 address = re.sub('[()]', '',a)
                
#             if item.find("span","result-price") is not None:
#                 rent = item.find("span","result-price").text
                
#             filewriter.writerow([post_id,tm,ht,address,rent,url])
#             #filewriter.writerow([post_id,tm,lat,long,house_type,address,rent,url])
        
